In [2]:
#Importing the packages i need for webscraping
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [17]:
#I want to research player statistics for the 2021/2022 SHL season.
url = "https://www.shl.se/statistik/tabell?season=2021&gameType=regular"
all_players = [] #In this list i will gather all the teamstatistics

In [18]:
#Getting all the urls for the different SHL teams.
data = requests.get(url)
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.rmss_t-stat-table')[0] #html tag is table, after the dot comes table name

links = [l.get("href") for l in standings_table.find_all('a')]
links = [l for l in links if '/lag/' in l]
team_urls = [f"http://shl.se{l}/statistics" for l in links] #Adding statistics after because thats the page i want
#access to.

print(team_urls)

['http://shl.se/lag/ee93-ee93uy4oW/statistics', 'http://shl.se/lag/1a71-1a71gTHKh/statistics', 'http://shl.se/lag/50e6-50e6DYeWM/statistics', 'http://shl.se/lag/087a-087aTQv9u/statistics', 'http://shl.se/lag/fe02-fe02mf1FN/statistics', 'http://shl.se/lag/752c-752c12zB7Z/statistics', 'http://shl.se/lag/82eb-82ebmgaJ8/statistics', 'http://shl.se/lag/9541-95418PpkP/statistics', 'http://shl.se/lag/259b-259bYGVIp/statistics', 'http://shl.se/lag/1ab8-1ab8bfj7N/statistics', 'http://shl.se/lag/41c4-41c4BiYZU/statistics', 'http://shl.se/lag/8e6f-8e6fUXJvi/statistics', 'http://shl.se/lag/2459-2459QTs1f/statistics', 'http://shl.se/lag/31d1-31d1NbSlR/statistics']


In [ ]:
#Now as you can see above we got all the different team links.
import time
#Getting all the statistics i want in this loop:
for team_url in team_urls:
    data = requests.get(team_url)
    soup = BeautifulSoup(data.text)
    team_name = soup.select('header.rmss_c-squad__team-header-name-h')[0] #I want the team name for my dataframe
    team_name = team_name.string #How to get the text between the tags out.
    players = pd.read_html(data.text, match="Spelare")[0] #Letting pandas figure out how to pull the whole statistics 
    #table from the site
    player_data = players
    player_data['team'] = team_name #Adding the team name to the dataframe
    all_players.append(player_data)
    time.sleep(2) #Using time.sleep to be nice to the site

In [ ]:
df = pd.concat(all_players) #Making the list into one big dataframe
df.columns = [c.lower() for c in df.columns] #Making all the columns be in lowcaps.
df.to_csv("players.csv", index=False) #Exporting to csv-file for further manipulation in R